In [49]:
import requests
import pandas as pd
import numpy as np
import json
import datetime
from dotenv import load_dotenv
import os

load_dotenv()
token = os.getenv('trakt_token')
cid = os.getenv('trakt_cid')

headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer ' + token,
  'trakt-api-version': '2',
  'trakt-api-key': cid
}

In [3]:
ratings = pd.read_csv('trakt-letterboxd/ratings.csv', header=0)
ratings['Rating'] = ratings['Rating']*2

w = requests.get('https://api.trakt.tv/sync/watched/movies/', headers=headers)
df = w.json()

In [4]:
imdb = []
titles = []
slug = []
for i in df:
    imdb.append(i['movie']['ids']['imdb'])
    titles.append(i['movie']['title'])
    slug.append(i['movie']['ids']['slug'])
    
it = pd.DataFrame(data={'imdb':imdb, 'titles':titles, 'slug':slug})
rt = pd.merge(it, ratings, left_on='titles', right_on='Name', how='inner')
rt.drop(['Date', 'Year', 'Letterboxd URI', 'Name'], axis=1, inplace=True)
rt.head()

imdb           titles                  slug  Rating
0   tt0468569  The Dark Knight  the-dark-knight-2008    10.0
1  tt12221748         Becoming   becoming-2020-05-06     8.0
2   tt0169547  American Beauty  american-beauty-1999    10.0
3   tt0209144          Memento          memento-2000     6.0
4   tt0075314      Taxi Driver      taxi-driver-1976     8.0

In [5]:
# data for POST ratings
dt = []
for i, r in rt.iterrows():
    dt.append(dict(rating=r['Rating'], ids=dict(imdb=r['imdb'])))

dt = dict(movies=dt)

In [75]:
add_rt = requests.post('https://api.trakt.tv/sync/ratings', headers=headers, data=json.dumps(dt))
add_rt.status_code

201

```
{
    "movies": [
        {
            "rated_at": "2014-09-01T09:10:11.000Z",
            "rating": 5,
            "title": "Batman Begins",
            "year": 2005,
            "ids": {
                "trakt": 1,
                "slug": "batman-begins-2005",
                "imdb": "tt0372784",
                "tmdb": 272
            }
        },
        {
            "rating": 10,
            "ids": {
                "imdb": "tt0000111"
            }
        }
    ]
}
```

In [25]:
rec = requests.get('https://api.trakt.tv/recommendations/movies', headers=headers)

rec_mv = rec.json()

In [34]:
rt

imdb                      titles                             slug  \
0     tt0468569             The Dark Knight             the-dark-knight-2008   
1    tt12221748                    Becoming              becoming-2020-05-06   
2     tt0169547             American Beauty             american-beauty-1999   
3     tt0209144                     Memento                     memento-2000   
4     tt0075314                 Taxi Driver                 taxi-driver-1976   
..          ...                         ...                              ...   
97    tt8613070  Portrait of a Lady on Fire  portrait-of-a-lady-on-fire-2019   
98    tt8404614               The Two Popes               the-two-popes-2019   
99    tt8637428                The Farewell                the-farewell-2019   
100   tt9683478              The Half of It              the-half-of-it-2020   
101   tt6285944                  The Banker                  the-banker-2020   

     Rating  
0      10.0  
1       8.0  
2      10.0  
3       6.0  
4       8.0  
..      ...  
97     10.0  
98     10.0  
99      7.0  
100     5.0  
101     8.0  

[102 rows x 4 columns]

In [35]:
stats = requests.get('https://api.trakt.tv/movies/portrait-of-a-lady-on-fire-2019/stats', headers=headers)
stats.json()

{'watchers': 5482,
 'plays': 7310,
 'collectors': 8761,
 'comments': 19,
 'lists': 3992,
 'votes': 1025}

In [37]:
omdb_api = os.getenv('omdb_api')
url = 'http://www.omdbapi.com/?apikey={}&i=tt8613070'.format('omdb_api')
o = requests.get(url)
o.status_code

200

In [38]:
o.json()

{'Title': 'Portrait of a Lady on Fire',
 'Year': '2019',
 'Rated': 'R',
 'Released': '14 Feb 2020',
 'Runtime': '122 min',
 'Genre': 'Drama, Romance',
 'Director': 'Céline Sciamma',
 'Writer': 'Céline Sciamma (screenplay)',
 'Actors': 'Noémie Merlant, Adèle Haenel, Luàna Bajrami, Valeria Golino',
 'Plot': 'On an isolated island in Brittany at the end of the eighteenth century, a female painter is obliged to paint a wedding portrait of a young woman.',
 'Language': 'French, Italian',
 'Country': 'France',
 'Awards': 'Nominated for 1 Golden Globe. Another 43 wins & 124 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BNjgwNjkwOWYtYmM3My00NzI1LTk5OGItYWY0OTMyZTY4OTg2XkEyXkFqcGdeQXVyODk4OTc3MTY@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.2/10'},
  {'Source': 'Metacritic', 'Value': '95/100'}],
 'Metascore': '95',
 'imdbRating': '8.2',
 'imdbVotes': '32,080',
 'imdbID': 'tt8613070',
 'Type': 'movie',
 'DVD': 'N/A',
 'BoxOffice': 'N/A',
 'P

In [8]:
rt.to_csv('ratings.csv')

In [19]:
w = requests.post('https://api.trakt.tv/sync/ratings', headers=headers, 
                  data=json.dumps(dict(movie=[dict(rating=7.0), dict(id=dict(imdb='tt0058625'))])))
w.status_code

201

In [20]:
h = requests.post('https://api.trakt.tv/sync/history', headers=headers,
                 data=json.dumps(dict(ids=dict(imdb='tt0058625'))))
h.status_code

201

In [45]:
# add new movies
ids = ['tt8290698']
rts = [8]

dt = []
now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for i in ids:
    dt.append(dict(watched_at=now, ids=dict(imdb=i)))

dt = dict(movies=dt)

h = requests.post('https://api.trakt.tv/sync/history', headers=headers, 
                  data=json.dumps(dt))

dt = []
for i, r in zip(ids, rts):
    dt.append(dict(rating=r, ids=dict(imdb=i)))

dt = dict(movies=dt)

r = requests.post('https://api.trakt.tv/sync/ratings', headers=headers, 
                  data=json.dumps(dt))